In [ ]:
!pip install -q biopython

%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 2.3MB 8.6MB/s 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = '/content/drive/MyDrive/data/chip-seq/'
LOG_DIR = '/content/drive/MyDrive/dna-nn/results/'

In [ ]:
!cp -r '/content/drive/MyDrive/dna-nn/dna_nn' .
!ls dna_nn

dataset.py  download.py  load.py  model.py


In [ ]:
import gc

import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras

from dna_nn.dataset import motif_discovery, motif_discovery_raw
from dna_nn.model import models, evaluate

# 2D features

In [ ]:
x_shape, train_ds, validation_data, test_ds = motif_discovery(DATA_DIR + 'motif_discovery_train.fasta', 
                                                              DATA_DIR + 'motif_discovery_test.fasta')

In [ ]:
models

In [ ]:
key = 'cnn_zeng_4_conv2d_l2'
dataset = 'motif_discovery'

keras.backend.clear_session()
model = models[key](x_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
model.summary()

In [ ]:
csv_path = LOG_DIR + f'{key}-{dataset}-dynamics.csv'
model_path = LOG_DIR + f'{key}-{dataset}.h5'

callbacks = [
    keras.callbacks.CSVLogger(csv_path),
    keras.callbacks.LambdaCallback(
        on_epoch_end=lambda epoch, logs: gc.collect(),
        # on_train_end=lambda logs: model.save(model_path)
    ),
    keras.callbacks.ModelCheckpoint(model_path, save_best_only=True)
]

history = model.fit(train_ds, epochs=50, validation_data=validation_data,
                    callbacks=callbacks, verbose=3)

In [ ]:
model = keras.models.load_model(model_path)
test_loss, test_accuracy = model.evaluate(test_ds, verbose=3)

y_score = model.predict(test_ds)
y_true = [int(y) for x, y in test_ds.unbatch()]
evaluate(model, history, test_accuracy, y_score, y_true, LOG_DIR, key, dataset)

In [ ]:
models

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve
import numpy as np

dataset = 'motif_discovery'
key = 'cnn_zeng_4_conv2d'
model_path = LOG_DIR + f'{key}-motif_discovery.h5'

model = keras.models.load_model(model_path)
test_loss, test_accuracy = model.evaluate(test_ds, verbose=3)

y_score = model.predict(test_ds)
y_true = [int(y) for x, y in test_ds.unbatch()]

precision, recall, thresholds = precision_recall_curve(y_true, y_score)
pr = pd.DataFrame({
    'precision': precision,
    'recall': recall,
    'thresholds': np.append(thresholds, np.nan)
})
pr.to_csv(LOG_DIR + f'{key}-{dataset}-pr.csv', index=False)

# 1D features

In [ ]:
x_shape, train_ds, validation_data, test_ds = motif_discovery(DATA_DIR + 'motif_discovery_train.fasta', 
                                                              DATA_DIR + 'motif_discovery_test.fasta',
                                                              3, 2, False)

In [ ]:
models

In [ ]:
key = 'deepram_recurrent_onehot'
dataset = 'motif_discovery'

keras.backend.clear_session()
model = models[key](x_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
model.summary()

In [ ]:
csv_path = LOG_DIR + f'{key}-{dataset}-dynamics.csv'
model_path = LOG_DIR + f'{key}-{dataset}.h5'

callbacks = [
    keras.callbacks.CSVLogger(csv_path),
    keras.callbacks.LambdaCallback(
        on_epoch_end=lambda epoch, logs: gc.collect(),
        # on_train_end=lambda logs: model.save(model_path)
    ),
    keras.callbacks.ModelCheckpoint(model_path, save_best_only=True)
]

history = model.fit(train_ds, epochs=50, validation_data=validation_data,
                    callbacks=callbacks, verbose=3)

In [ ]:
model = keras.models.load_model(model_path)
test_loss, test_accuracy = model.evaluate(test_ds, verbose=3)

y_score = model.predict(test_ds)
y_true = [int(y) for x, y in test_ds.unbatch()]
evaluate(model, history, test_accuracy, y_score, y_true, LOG_DIR, key, dataset)

# Raw features

In [ ]:
x_shape, train_ds, validation_data, test_ds = motif_discovery_raw(DATA_DIR + 'motif_discovery_train.fasta', 
                                                                  DATA_DIR + 'motif_discovery_test.fasta')

In [ ]:
models

In [ ]:
key = 'deepram_recurrent_embed'
dataset = 'motif_discovery'

keras.backend.clear_session()
model = models[key](x_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 101, 256)          25856     
_________________________________________________________________
dropout (Dropout)            (None, 101, 256)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 101, 64)           82176     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [ ]:
csv_path = LOG_DIR + f'{key}-{dataset}-dynamics.csv'
model_path = LOG_DIR + f'{key}-{dataset}.h5'

callbacks = [
    keras.callbacks.CSVLogger(csv_path),
    keras.callbacks.LambdaCallback(
        on_epoch_end=lambda epoch, logs: gc.collect(),
        # on_train_end=lambda logs: model.save(model_path)
    ),
    keras.callbacks.ModelCheckpoint(model_path, save_best_only=True)
]

history = model.fit(train_ds, epochs=50, validation_data=validation_data,
                    callbacks=callbacks, verbose=3)

In [ ]:
model = keras.models.load_model(model_path)
test_loss, test_accuracy = model.evaluate(test_ds, verbose=3)

y_score = model.predict(test_ds)
y_true = [int(y) for x, y in test_ds.unbatch()]
evaluate(model, history, test_accuracy, y_score, y_true, LOG_DIR, key, dataset)